In [8]:
# https://discuss.pytorch.org/t/i-have-3-gpu-why-torch-cuda-device-count-only-return-1/7245/4
import torch
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

[autoreload of datamodule failed: Traceback (most recent call last):
  File "/home/user/miniconda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/user/miniconda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/user/miniconda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/user/miniconda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/user/miniconda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/user/miniconda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 266, in update_function
    setattr(old, name, getattr(new, name))
ValueError: set

__Python VERSION: 3.8.3 (default, May 19 2020, 18:47:26) 
[GCC 7.3.0]
__pyTorch VERSION: 1.7.0
__CUDA VERSION
/usr/bin/sh: 1: nvcc: not found
__CUDNN VERSION: 8003
__Number CUDA Devices: 2
__Devices
Active CUDA Device: GPU 1
Available devices  2
Current cuda device  1


In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import main
import random
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser

wandb: Currently logged in as: chucooleg (use `wandb login --relogin` to force relogin)


In [4]:
# arguments
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

args.data_path = '../Raw_Datasets/SET/4Attr-3Vals-1LenMultiplier-1NestDepth-2048Train-2048Val-2048Test.json'
args.config_path = 'config.json'
args.generate_full_matrix = True
args.checkpoint_dir = 'checkpoints'
args.mode = 'train'
args.gpu = -1
args.approve_before_training = False

In [5]:
trainmodule, game_datamodule = main.main(args)

Global seed set to 42
/home/user/miniconda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Checkpoint directory checkpoints/ContrastiveLearning-cardgame-SETShattering/20210316-213741-SET;attr4-val3-1LenMul-nest1;contrastive;d_model128;dot-product;params715.52K exists and is not empty.
  warnings.warn(*args, **kwargs)
/home/user/miniconda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: You requested multiple GPUs but did not specify a backend, e.g. `Trainer(accelerator="dp"|"ddp"|"ddp2")`. Setting `accelerator="ddp_spawn"` for you.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores


---------data----------
num_attributes : 4
num_attr_vals : 3
nest_depth_int : 1
key_support_size : 81
query_length_multiplier : 1
max_len_q : 412
len_k : 4
train_key_datapoints length : 2048
val_key_datapoints length : 2048
test_key_datapoints length : 2048
train_gt_idxs length : 2048
val_gt_idxs length : 2048
test_gt_idxs length : 2048
train_tokens length : 2048
val_tokens length : 2048
test_tokens length : 2048
vocab_size : 22
( : 12
) : 13
NULL : 14
SEP : 15
SOS : 16
EOS : 17
PAD : 18
PLH : 19
& : 20
| : 21
-----------------------
----------hparams----------
seed : 42
batch_size : 512
max_epochs : 10
val_every_n_epoch : 1
d_model : 128
embed_dropout : 0.0
model : contrastive
d_ff : 1024
num_heads : 1
N_enc : 1
N_dec : 1
attn_wt_tying_scheme : untie_QKVO
attn_wt_dropout : 0.0
heads_dropout : 0.0
pff_dropout : 0.0
representation_pos : 0
dotproduct_bottleneck : True
normalize_dotproduct : True
contrastive_use_infoNCE : True
loss_temperature_const : 1.0
loss_smoothing_const : 0.1
nonlin

KeyboardInterrupt: 

In [7]:
16e6 / 30720 * (15*60) / 3600

130.20833333333334

In [ ]:
AssertionError: Caught AssertionError in DataLoader worker process 0.